## First we register a test user in SQuaSH and get an access token 
You may be running the [squash-restful-api](https://github.com/lsst-sqre/squash-rest-api) app locally, in this case use `SQUASH_API_URL=http://localhost:5000`, or you could use the deployed demo at https://squash-restful-api-demo.lsst.codes if available.

You can check the API documentation at https://sqr-009.lsst.io/#the-squash-restful-api


In [ ]:
#SQUASH_API_URL = "http://localhost:5000"
SQUASH_API_URL = "https://squash-restful-api-demo.lsst.codes"

In [ ]:
import requests

user={'username': 'testuser', 'password': 'testpwd'}
r = requests.post(SQUASH_API_URL + "/register", json=user)
r.json()


In [ ]:
r = requests.post(SQUASH_API_URL + "/auth", json=user)
r.json()

Add the access token into the authorization header for the next requests.

In [ ]:
headers = dict()
headers['Authorization'] = 'JWT ' + r.json()['access_token']

## We'll use a test verification job 
You can reproduce this verification job document with the following command:

`$ dispatch_verify.py --test --env jenkins --lsstsw $(pwd) Cfht_output_r.json --write verify_job.json`

where `Cfht_output_r.json` is an output from `validate_drp`.

In [ ]:
import json
with open("data/verify_job.json", "r") as f:
     data = json.load(f)

A verification job is a container for measurements, metadata, data blobs as well as metric definitions and specifications. Metric definitions and specifications are pre-loaded to SQuaSH and only measurements, metadata and data blobs must be sent to SQuaSH each time a new verification job is registered. For the purpose of testing the API we will exercise loading metric definitions and specifications here too.

In [ ]:
data.keys()

In [ ]:
data['blobs'][0].keys()

## Loading metric definitions


In [ ]:
metrics={'metrics': data['metrics']}
r = requests.post(SQUASH_API_URL + "/metrics", json=metrics, headers=headers)
r.json()


## Updating a metric definition

In [ ]:
metric = data['metrics'][0]
metric

In [ ]:
r = requests.put(SQUASH_API_URL + "/metric/{}".format(metric['name']), json=metric, headers=headers)
r.json()

In [ ]:
r = requests.get(SQUASH_API_URL + "/metric/{}".format(metric['name']))
r.json()

## Loading specifications

In [ ]:
specs={'specs': data['specs']}
r = requests.post(SQUASH_API_URL + "/specs", json=specs, headers=headers)
r.json()


## Update a specification

In [ ]:
spec = data['specs'][0]
spec

In [ ]:
r = requests.put(SQUASH_API_URL + "/spec/{}".format(spec['name']), json=spec, headers=headers)
r.json()

In [ ]:
r = requests.get(SQUASH_API_URL + "/spec/{}".format(spec['name']))
r.json()

## Create a job



In addition to measurements and blobs, add job metatada which consists of (arbitrary) metadata, packages and environment metadata.

Specify the dataset for this job

In [ ]:
data['meta']['env']['ci_dataset']='validation_data_cfht'


In [ ]:
%%time
r = requests.post(SQUASH_API_URL + "/auth", json=user)
headers['Authorization'] = 'JWT ' + r.json()['access_token']
job = {'measurements': data['measurements'], 'meta': data['meta'], 'blobs': data['blobs']}
r = requests.post(SQUASH_API_URL + "/job", json=job, headers=headers)
r.json()

In [ ]:
%%time
r = requests.post(SQUASH_API_URL + "/auth", json=user)
headers['Authorization'] = 'JWT ' + r.json()['access_token']
job = {'measurements': data['measurements'], 'meta': data['meta'], 'blobs': []}
r = requests.post(SQUASH_API_URL + "/job", json=job, headers=headers)
r.json()

## Finally we generate some verification jobs for testing varying the enviroment metadata as needed



Job metatada consists of (arbitrary) metadata, packages and environment metadata

In [ ]:
import random
import time
datasets=['cfht', 'decam', 'hsc']
for ci_id in range(10):
    data['meta']['env']['ci_id']=str(ci_id)
    data['meta']['env']['ci_url']="https://example.com/{}".format(ci_id)
    data['meta']['env']['ci_dataset']=random.choice(datasets)
    job = {'measurements': data['measurements'], 'meta': data['meta'], 'blobs': data['blobs']}
    # sleep 15s to have the jobs created at different times
    time.sleep(15)
    r = requests.post(SQUASH_API_URL + "/job", json=job, headers=headers)
    r.json()


## You can retrieve a job from SQuaSH using

In [ ]:
r = requests.get(SQUASH_API_URL + "/job/1")
#r.json()